In [1]:
import os
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
SCORES_DIR = "data/scores"
box_scores = os.listdir(SCORES_DIR)

In [3]:
len(box_scores)

7581

In [4]:
box_scores = [os.path.join(SCORES_DIR, f) for f in box_scores if f.endswith(".html")]

In [5]:
def parse_html(box_score):
    with open(box_score) as f:
        html = f.read()

    #cleaning html
    soup = BeautifulSoup(html)
    [s.decompose() for s in soup.select("tr.over_header")]
    [s.decompose() for s in soup.select("tr.thead")]
    return soup

In [6]:
def read_linescore(soup):
    line_score = pd.read_html(str(soup), attrs = {"id": "line_score"} )[0]
    cols = list(line_score.columns)
    cols[0] = "team"
    cols[-1] = "total"
    line_score.columns = cols
    line_score = line_score[["team", "total"]]
    return line_score


In [7]:
def read_stats(soup, team, stat_type):
    df = pd.read_html(str(soup), attrs = {"id": f'box-{team}-game-{stat_type}'}, index_col = 0)[0]
    df = df.apply(pd.to_numeric, errors = "coerce") #converts string stats to NaN
    return df

In [8]:
def read_season_info(soup):
    nav = soup.select("#bottom_nav_container")[0]
    hrefs = [a["href"] for a in nav.find_all("a")]
    season = os.path.basename(hrefs[1]).split("_")[0]
    return season


In [9]:
base_cols = None
games = []
box_score = box_scores[0]

for box_score in box_scores:
    soup = parse_html(box_score)
    line_score = read_linescore(soup)
    teams = list(line_score["team"])


    summaries = []
    for team in teams:
        basic = read_stats(soup, team, "basic")
        advanced = read_stats(soup, team, "advanced")

        totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]])
        maxes = pd.concat([basic.iloc[:-1,:].max(), advanced.iloc[:-1,:].max()])
        maxes.index = maxes.index.str.lower() + "_max"
        totals.index = totals.index.str.lower()#making sure pd treats these as different columns

        summary = pd.concat([totals, maxes])

        if base_cols is None:
            base_cols = list(summary.index.drop_duplicates(keep = "first"))
            base_cols = [b for b in base_cols if "bpm" not in b]

        summary = summary[base_cols]

        summaries.append(summary)

    summary = pd.concat(summaries, axis = 1).T

    game = pd.concat([summary, line_score], axis = 1)


    game["home"] = [0,1]
    game_opp = game.iloc[::-1].reset_index()
    game_opp.columns += "_opp"

    full_game = pd.concat([game, game_opp], axis = 1)

    full_game["season"] = read_season_info(soup)

    full_game["date"] = os.path.basename(box_score)[:8]
    full_game["date"] = pd.to_datetime(full_game["date"], format = "%Y%m%d")

    full_game["winner"] = full_game["total"] > full_game["total_opp"]
    games.append(full_game)
    if len(games) % 100 == 0:
        print(f"Processed {len(games)} / {len(box_scores)} games")


/var/folders/rt/vlxzyc6d2mggxf0fjtgxlls80000gn/T/ipykernel_62451/715654178.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  line_score = pd.read_html(str(soup), attrs = {"id": "line_score"} )[0]
/var/folders/rt/vlxzyc6d2mggxf0fjtgxlls80000gn/T/ipykernel_62451/360467897.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(soup), attrs = {"id": f'box-{team}-game-{stat_type}'}, index_col = 0)[0]
/var/folders/rt/vlxzyc6d2mggxf0fjtgxlls80000gn/T/ipykernel_62451/360467897.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(soup), attrs = {"id": f'box-{team}-game-{stat_type}'}, i

Processed 5600 / 7581 games


/var/folders/rt/vlxzyc6d2mggxf0fjtgxlls80000gn/T/ipykernel_62451/715654178.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  line_score = pd.read_html(str(soup), attrs = {"id": "line_score"} )[0]
/var/folders/rt/vlxzyc6d2mggxf0fjtgxlls80000gn/T/ipykernel_62451/360467897.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(soup), attrs = {"id": f'box-{team}-game-{stat_type}'}, index_col = 0)[0]
/var/folders/rt/vlxzyc6d2mggxf0fjtgxlls80000gn/T/ipykernel_62451/360467897.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(soup), attrs = {"id": f'box-{team}-game-{stat_type}'}, i

Processed 5700 / 7581 games


/var/folders/rt/vlxzyc6d2mggxf0fjtgxlls80000gn/T/ipykernel_62451/715654178.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  line_score = pd.read_html(str(soup), attrs = {"id": "line_score"} )[0]
/var/folders/rt/vlxzyc6d2mggxf0fjtgxlls80000gn/T/ipykernel_62451/360467897.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(soup), attrs = {"id": f'box-{team}-game-{stat_type}'}, index_col = 0)[0]
/var/folders/rt/vlxzyc6d2mggxf0fjtgxlls80000gn/T/ipykernel_62451/360467897.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(soup), attrs = {"id": f'box-{team}-game-{stat_type}'}, i

...

/var/folders/rt/vlxzyc6d2mggxf0fjtgxlls80000gn/T/ipykernel_62451/715654178.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  line_score = pd.read_html(str(soup), attrs = {"id": "line_score"} )[0]
/var/folders/rt/vlxzyc6d2mggxf0fjtgxlls80000gn/T/ipykernel_62451/360467897.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(soup), attrs = {"id": f'box-{team}-game-{stat_type}'}, index_col = 0)[0]
/var/folders/rt/vlxzyc6d2mggxf0fjtgxlls80000gn/T/ipykernel_62451/360467897.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(soup), attrs = {"id": f'box-{team}-game-{stat_type}'}, i

Processed 7400 / 7581 games


/var/folders/rt/vlxzyc6d2mggxf0fjtgxlls80000gn/T/ipykernel_62451/715654178.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  line_score = pd.read_html(str(soup), attrs = {"id": "line_score"} )[0]
/var/folders/rt/vlxzyc6d2mggxf0fjtgxlls80000gn/T/ipykernel_62451/360467897.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(soup), attrs = {"id": f'box-{team}-game-{stat_type}'}, index_col = 0)[0]
/var/folders/rt/vlxzyc6d2mggxf0fjtgxlls80000gn/T/ipykernel_62451/360467897.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(soup), attrs = {"id": f'box-{team}-game-{stat_type}'}, i

Processed 7500 / 7581 games


/var/folders/rt/vlxzyc6d2mggxf0fjtgxlls80000gn/T/ipykernel_62451/715654178.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  line_score = pd.read_html(str(soup), attrs = {"id": "line_score"} )[0]
/var/folders/rt/vlxzyc6d2mggxf0fjtgxlls80000gn/T/ipykernel_62451/360467897.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(soup), attrs = {"id": f'box-{team}-game-{stat_type}'}, index_col = 0)[0]
/var/folders/rt/vlxzyc6d2mggxf0fjtgxlls80000gn/T/ipykernel_62451/360467897.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(soup), attrs = {"id": f'box-{team}-game-{stat_type}'}, i

In [15]:
games_df = pd.concat(games, ignore_index = True)
games_df.to_csv("nbagames.csv")

In [19]:
games_df.head()

,mp,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,winner
0,240.0,240.0,38.0,72.0,0.528,16.0,37.0,0.432,18.0,21.0,...,26.2,26.8,155.0,123.0,MIA,107,1,2023,2022-11-07,True
1,240.0,240.0,40.0,84.0,0.476,14.0,39.0,0.359,13.0,15.0,...,41.0,37.3,160.0,121.0,POR,110,0,2023,2022-11-07,False
2,240.0,240.0,41.0,78.0,0.526,8.0,24.0,0.333,15.0,19.0,...,28.6,41.1,250.0,125.0,DAL,90,1,2023,2022-12-14,True
3,240.0,240.0,29.0,74.0,0.392,13.0,38.0,0.342,19.0,26.0,...,12.6,33.0,183.0,110.0,CLE,105,0,2023,2022-12-14,False
4,240.0,240.0,37.0,87.0,0.425,7.0,33.0,0.212,32.0,35.0,...,20.0,32.3,226.0,117.0,TOR,126,1,2023,2022-12-07,False


In [22]:
[g.shape[1] for g in games if g.shape[1] != 154]



[]